This notebook is sectioned by heading. There is a 'Tyler' section and a 'David' section. If we put our stuff in only our own section, we should be able to work on the same notebook with no issues.

Things to investigate:

- How many investments are by doctors who have multiple investments? (Tyler, done)

- Which states / zip codes are associated with the most investments? Does affluence of business zipcode correlate with doctor investment? (David, in progress)

- Which doctors have invested the most money? In the most companies?

- Which investments were made during or prior to 2015? What is the total sum of these different groups of investments?

- Which businesses hold the most investments from doctors? Are they geographically close to each other?

In [21]:
# library imports
import pandas as pd
import numpy as np

In [22]:
#     GNRL is 6 gigabytes - too big
# record_general = pd.read_csv('data/OP_DTL_GNRL_PGYR2015_P01172017.csv')
#     RSRCH has hundred of bad columns - not good for class
# record_research = pd.read_csv('data/OP_DTL_RSRCH_PGYR2015_P01172017.csv')
#     REMOVED has very little data, so we're ignoring it for now
# record_removed = pd.read_csv('data/OP_REMOVED_DELETED_PGYR2015_P01172017.csv')

In [23]:
record_ownership = pd.read_csv('data/OP_DTL_OWNRSHP_PGYR2015_P01172017.csv')
record_ownership.shape

(4319, 29)

In [24]:
record_ownership.head()

,Change_Type,Physician_Profile_ID,Physician_First_Name,Physician_Middle_Name,Physician_Last_Name,Physician_Name_Suffix,Recipient_Primary_Business_Street_Address_Line1,Recipient_Primary_Business_Street_Address_Line2,Recipient_City,Recipient_State,...,Value_of_Interest,Terms_of_Interest,Submitting_Applicable_Manufacturer_or_Applicable_GPO_Name,Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID,Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name,Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_State,Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Country,Dispute_Status_for_Publication,Interest_Held_by_Physician_or_an_Immediate_Family_Member,Payment_Publication_Date
0,UNCHANGED,333118,Kelvin,NaN,Higa,NaN,205 E River Park Circle,Ste 460,Fresno,CA,...,733.26,Not Publicly Traded - Common. 409a estimated ...,"Endogastric Solutions, Inc",100000010963,"Endogastric Solutions, Inc",WA,United States,No,Physician Covered Recipient,01/17/2017
1,UNCHANGED,270405,Spencer,NaN,Holover,NaN,3003 New Hyde Park Rd,Ste 307,New Hyde Park,NY,...,3070.52,Not Publicly Traded - Preferred. 409a estim...,"Endogastric Solutions, Inc",100000010963,"Endogastric Solutions, Inc",WA,United States,No,Physician Covered Recipient,01/17/2017
2,UNCHANGED,4927,Shawn,NaN,Garber,NaN,3003 New Hyde Park Rd,Ste 307,New Hyde Park,NY,...,12282.66,Not Publicly Traded - Preferred. 409a estim...,"Endogastric Solutions, Inc",100000010963,"Endogastric Solutions, Inc",WA,United States,No,Physician Covered Recipient,01/17/2017
3,UNCHANGED,132587,Yu-Guang,NaN,He,NaN,5323 Harry Hines Blvd.,NaN,Dallas,TX,...,330940.54,Direct ownership,"Vital Art and Science, LLC",100000005370,"Vital Art and Science, LLC",TX,United States,No,Physician Covered Recipient,01/17/2017
4,UNCHANGED,885629,Allan,NaN,Deutsch,NaN,345 E 80th St,NaN,New York,NY,...,1548787.00,fifty percent owner in reporting entity,ESSENTIAL DENTAL SYSTEMS INCORPORATED,100000005369,ESSENTIAL DENTAL SYSTEMS INCORPORATED,NJ,United States,No,Physician Covered Recipient,01/17/2017


## Tyler: How many investments are by doctors who have multiple investments?

In [25]:
doctors = pd.DataFrame([])
doctors['ID'] = record_ownership['Physician_Profile_ID']
doctors['First_Name'] = record_ownership['Physician_First_Name']
doctors['Middle_Name'] = record_ownership['Physician_Middle_Name']
doctors['Last_Name'] = record_ownership['Physician_Last_Name']

doctors.shape

(4319, 4)

In [26]:
doctors.head()

,ID,First_Name,Middle_Name,Last_Name
0,333118,Kelvin,NaN,Higa
1,270405,Spencer,NaN,Holover
2,4927,Shawn,NaN,Garber
3,132587,Yu-Guang,NaN,He
4,885629,Allan,NaN,Deutsch


In [27]:
# sort by IDs (unneccessary)
doctors_sorted = doctors.sort_values('ID')
doctors_sorted.head()

,ID,First_Name,Middle_Name,Last_Name
3815,359,Douglas,K,Blacklidge
3919,571,Thomas,W,Childress
4204,869,Szilard,NaN,Kiss
3864,1073,Kevin,NaN,Gancarczyk
732,1182,ERIC,A,PIZZA


In [28]:
# count the unique IDs
uniq = len(doctors.ID.unique())
total = doctors.shape[0]

print str(uniq) + " unique physicians in data set of " + str(total) + " observations."
print str(total - uniq) + " observations are associated with physicians with 2 or more investments."

3870 unique physicians in data set of 4319 observations.
449 observations are associated with physicians with 2 or more investments.


## David: Which states / zip codes are associated with the most investments? Does affluence of business zipcode correlate with doctor investment?

In [10]:
record_ownership.columns.values
len(record_ownership)
#record_ownership['Program_Year'].unique()
#record_ownership['Terms_of_Interest'].head(25)
#record_ownership['Submitting_Applicable_Manufacturer_or_Applicable_GPO_Name'].head(25)
#record_ownership['Interest_Held_by_Physician_or_an_Immediate_Family_Member'].head(25)

4319

In [92]:
Geography = record_ownership[['Recipient_Primary_Business_Street_Address_Line1', 'Recipient_Primary_Business_Street_Address_Line2', 
                             'Recipient_City', 'Recipient_State', 'Recipient_Zip_Code', 'Recipient_Country', 'Recipient_Province', 
                             'Recipient_Postal_Code', 'Physician_Primary_Type', 'Physician_Specialty', 'Total_Amount_Invested_USDollars',
                             'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_State',
                             'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Country',
                             'Dispute_Status_for_Publication'
                            ]]

Geography.head()


,Recipient_Primary_Business_Street_Address_Line1,Recipient_Primary_Business_Street_Address_Line2,Recipient_City,Recipient_State,Recipient_Zip_Code,Recipient_Country,Recipient_Province,Recipient_Postal_Code,Physician_Primary_Type,Physician_Specialty,Total_Amount_Invested_USDollars,Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_State,Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Country,Dispute_Status_for_Publication
0,205 E River Park Circle,Ste 460,Fresno,CA,93720,United States,NaN,NaN,Medical Doctor,Allopathic & Osteopathic Physicians|Surgery,2500.00,WA,United States,No
1,3003 New Hyde Park Rd,Ste 307,New Hyde Park,NY,11042,United States,NaN,NaN,Medical Doctor,Other Service Providers/ Specialist,24999.69,WA,United States,No
2,3003 New Hyde Park Rd,Ste 307,New Hyde Park,NY,11042,United States,NaN,NaN,Medical Doctor,Other Service Providers/ Specialist,99998.74,WA,United States,No
3,5323 Harry Hines Blvd.,NaN,Dallas,TX,75390-9057,United States,NaN,NaN,Medical Doctor,Allopathic & Osteopathic Physicians|Ophthalmology,10000.00,TX,United States,No
4,345 E 80th St,NaN,New York,NY,10075,United States,NaN,NaN,Doctor of Dentistry,Dental Providers|Dentist|Oral and Maxillofacia...,1548787.00,NJ,United States,No


In [93]:
#Geography.groupby('Dispute_Status_for_Publication').count()
Geography['Recipient_Country'].unique()

array(['United States', 'United States Minor Outlying Islands',
       'United Arab Emirates'], dtype=object)

In [94]:
#import urllib
#url = 'https://www.irs.gov/pub/irs-soi/14zpallagi.csv'
#response = urllib.urlopen(url)
#csv = response.read()
#with open('data/Income.csv', 'wb') as f:
#    f.write(csv)
    
#Documentation at https://www.irs.gov/pub/irs-soi/14zpdoc.doc
income = pd.read_csv('data/Income.csv')
income.head()

,STATEFIPS,STATE,zipcode,agi_stub,N1,mars1,MARS2,MARS4,PREP,N2,...,N10300,A10300,N85530,A85530,N85300,A85300,N11901,A11901,N11902,A11902
0,1,AL,0,1,850050.0,481840.0,115070.0,240450.0,479900.0,1401930.0,...,389850.0,324575.0,0.0,0.0,0.0,0.0,62690.0,47433.0,744910.0,1964826.0
1,1,AL,0,2,491370.0,200750.0,150290.0,125560.0,281350.0,1016010.0,...,397110.0,950446.0,0.0,0.0,0.0,0.0,70780.0,101969.0,413790.0,1177400.0
2,1,AL,0,3,259540.0,75820.0,142970.0,34070.0,156720.0,589190.0,...,250230.0,1319641.0,0.0,0.0,0.0,0.0,62170.0,132373.0,192050.0,538160.0
3,1,AL,0,4,164840.0,26730.0,125410.0,10390.0,99750.0,423300.0,...,163580.0,1394913.0,0.0,0.0,0.0,0.0,45120.0,124048.0,115470.0,375882.0
4,1,AL,0,5,203650.0,18990.0,177070.0,5860.0,122670.0,565930.0,...,203050.0,3655700.0,610.0,135.0,270.0,66.0,81180.0,387298.0,114380.0,448442.0


In [96]:
income = income[['STATE', 'zipcode', 'agi_stub']]
income = income[income['zipcode'] != 0]
#income.head()
income['zipcode'] = income['zipcode'].apply('{:0>5}'.format)
#income[income['zipcode'].apply(lambda x: len(str(x))) == 5]
Geography['zipcode'] = Geography['Recipient_Zip_Code'].apply(lambda s: str(s).split('-')[0]) #.str.split('-')

C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


## Tyler: Which doctors have invested the most money? In the most companies?

In [19]:
# get a view of the data we want
temp = record_ownership[["Physician_Profile_ID",
                         "Physician_First_Name",
                         "Physician_Middle_Name",
                         "Physician_Last_Name",
                         "Total_Amount_Invested_USDollars",
                         "Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID",
                         "Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name"]]


# build the dataframe out of that view
doc_invest = pd.DataFrame(temp, copy=True)
doc_invest = doc_invest.rename(columns={"Physician_Profile_ID":"Physician_ID",
                                        "Total_Amount_Invested_USDollars":"Total_Invesment",
                                        "Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID":"Company_ID",
                                        "Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name":"Company_Name"})
doc_invest.head()

,Physician_ID,Physician_First_Name,Physician_Middle_Name,Physician_Last_Name,Total_Invesment,Company_ID,Company_Name
0,333118,Kelvin,NaN,Higa,2500.00,100000010963,"Endogastric Solutions, Inc"
1,270405,Spencer,NaN,Holover,24999.69,100000010963,"Endogastric Solutions, Inc"
2,4927,Shawn,NaN,Garber,99998.74,100000010963,"Endogastric Solutions, Inc"
3,132587,Yu-Guang,NaN,He,10000.00,100000005370,"Vital Art and Science, LLC"
4,885629,Allan,NaN,Deutsch,1548787.00,100000005369,ESSENTIAL DENTAL SYSTEMS INCORPORATED


In [35]:
# Dictionary to hold the doctor:total_investment data
investments = {}

for doctor in doc_invest["Physician_ID"].unique():
    investments[doctor] = np.sum(doc_invest["Total_Invesment"].where(doc_invest["Physician_ID"] == doctor))

len(investments)

(214071, 50320.0)

This matches our observation above: 3870 unique physicians in data set of 4319 observations. Which of these 3870 doctors holds the greatest total investments?

In [36]:
# sort the docs
sorted_docs = sorted(investments.items(), # doctor_id, value
                     key=lambda x: x[1],  # sort on value
                     reverse=True)        # biggest to smallest

# grab the doctor names via the ID
for doc, total in sorted_docs[:10]:
    just_doc = doc_invest.where(doc_invest["Physician_ID"] == doc)
    doc_f = just_doc["Physician_First_Name"].iloc(0)
    doc_m = just_doc["Physician_Middle_Name"].iloc(0)
    doc_l = just_doc["Physician_Last_Name"].iloc(0)
    
    out_str = str(doc_f) + " " + str(doc_m) + " " + str(doc_l) + " has invested " + str(total) + " USD"
    print out_str

<pandas.core.indexing._iLocIndexer object at 0x000000000A62C4E0> <pandas.core.indexing._iLocIndexer object at 0x000000000A62C438> <pandas.core.indexing._iLocIndexer object at 0x000000000A62C630> has invested 199733425.11
<pandas.core.indexing._iLocIndexer object at 0x000000000A3AB780> <pandas.core.indexing._iLocIndexer object at 0x000000000A3AB518> <pandas.core.indexing._iLocIndexer object at 0x000000000A3AB550> has invested 10714540.75
<pandas.core.indexing._iLocIndexer object at 0x000000000A3AB400> <pandas.core.indexing._iLocIndexer object at 0x000000000A3AB828> <pandas.core.indexing._iLocIndexer object at 0x000000000A3ABEF0> has invested 6295918.17
<pandas.core.indexing._iLocIndexer object at 0x000000000A3AB470> <pandas.core.indexing._iLocIndexer object at 0x000000000A3AB5F8> <pandas.core.indexing._iLocIndexer object at 0x000000000A3ABF60> has invested 5719872.54
<pandas.core.indexing._iLocIndexer object at 0x000000000A8BE400> <pandas.core.indexing._iLocIndexer object at 0x000000000

In [41]:
for doc, total in sorted_docs[:10]:
    just_doc = doc_invest.where(doc_invest["Physician_ID"] == doc)
    print type(just_doc)

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
